# Example Model Servers with Seldon

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Setup-Cluster) with [Ambassador Ingress](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Ambassador) and [Seldon Core](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html#Install-Seldon-Core). Instructions [also online](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html).

In [1]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [16]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


In [3]:
import json

## Serve SKLearn Iris Model

In order to deploy SKLearn artifacts, we can leverage the [pre-packaged SKLearn inference server](https://docs.seldon.io/projects/seldon-core/en/latest/servers/sklearn.html).
The exposed API can follow either:

- The default Seldon protocol. 
- The [KFServing V2 protocol](https://docs.seldon.io/projects/seldon-core/en/latest/servers/sklearn.html##v2-kfserving-protocol-incubating).

For details on each of these protocols, you can check the [documentation section on API protocols](https://docs.seldon.io/projects/seldon-core/en/latest/graph/protocols.html#v2-kfserving-protocol).


### Default Seldon protocol

To deploy and start serving an SKLearn artifact using Seldon's default protocol, we can use a config like the one below:

In [25]:
%%writefile ../servers/sklearnserver/samples/iris.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1
    svcOrchSpec: 
      env: 
      - name: SELDON_LOG_LEVEL
        value: DEBUG

Overwriting ../servers/sklearnserver/samples/iris.yaml


We can then apply it to deploy it to our Kubernetes cluster.

In [26]:
!kubectl apply -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io/sklearn created


In [27]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn -o jsonpath='{.items[0].metadata.name}')

deployment "sklearn-default-0-classifier" successfully rolled out


Once it's deployed we can send our sklearn model requests

#### REST Requests

In [28]:
X=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0, 6.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/sklearn/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

{'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[9.912315378486718e-07, 0.0007015931307743852, 0.9992974156376878]]}, 'meta': {}}


In [29]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="sklearn",namespace="seldon")

In [30]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.6806334106409295
    values: 0.28830360482888695
    values: 0.8975654688578534
    values: 0.24493223527153596
  }
}

Response:
{'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.14449927217770223, 0.5104835080299303, 0.34501721979236755]}}, 'meta': {}}


#### gRPC Requests

In [31]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 4], 'values': [0.22157381246204433, 0.020827790786912548, 0.7633031884959527, 0.19217598516001178]}}}
Response:
{'meta': {}, 'data': {'names': ['t:0', 't:1', 't:2'], 'tensor': {'shape': [1, 3], 'values': [0.10758420949891327, 0.4796849750770796, 0.4127308154240072]}}}


In [32]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0,6.0]]}}' \
         -rpc-header seldon:sklearn -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)

{'meta': {}, 'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[9.912315378486718e-07, 0.0007015931307743852, 0.9992974156376878]]}}


And delete the model we deployed

In [33]:
!kubectl delete -f ../servers/sklearnserver/samples/iris.yaml

seldondeployment.machinelearning.seldon.io "sklearn" deleted


### KFServing V2 protocol

We can deploy a SKLearn artifact, exposing an API compatible with [KFServing's V2 Protocol](https://docs.seldon.io/projects/seldon-core/en/latest/servers/sklearn.html##v2-kfserving-protocol-incubating) by specifying the `protocol` of our `SeldonDeployment` as `kfserving`.
For example, we can consider the config below:

In [6]:
%%writefile ./resources/iris-sklearn-v2.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: sklearn
spec:
  name: iris
  protocol: kfserving
  predictors:
  - graph:
      children: []
      implementation: SKLEARN_SERVER
      modelUri: gs://seldon-models/sklearn/iris
      name: classifier
    name: default
    replicas: 1

Overwriting ./resources/iris-sklearn-v2.yaml


We can then apply it to deploy our model to our Kubernetes cluster.

In [2]:
!kubectl apply -f resources/iris-sklearn-v2.yaml

seldondeployment.machinelearning.seldon.io/sklearn created


In [3]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=sklearn -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "sklearn-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "sklearn-default-0-classifier" successfully rolled out


Once it's deployed, we can send inference requests to our model.
Note that, since it's using the KFServing's V2 Protocol, these requests will be different to the ones using the default Seldon Protocol.

In [4]:
import requests
import json

inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": [1, 4],
          "datatype": "FP32",
          "data": [[1, 2, 3, 4]]
        }
    ]
}

endpoint = "http://localhost:8003/seldon/seldon/sklearn/v2/models/infer"
response = requests.post(endpoint, json=inference_request)

print(json.dumps(response.json(), indent=2))
assert response.ok

{
  "model_name": "classifier",
  "model_version": "v1",
  "id": "5a0bce13-98e8-4588-9fc5-7c8b11140174",
  "parameters": null,
  "outputs": [
    {
      "name": "predict",
      "shape": [
        1
      ],
      "datatype": "FP32",
      "parameters": null,
      "data": [
        2
      ]
    }
  ]
}


Finally, we can delete the model we deployed.

In [5]:
!kubectl delete -f resources/iris-sklearn-v2.yaml

seldondeployment.machinelearning.seldon.io "sklearn" deleted


## Serve XGBoost Iris Model

In order to deploy XGBoost models, we can leverage the [pre-packaged XGBoost inference server](https://docs.seldon.io/projects/seldon-core/en/latest/servers/xgboost.html).
The exposed API can follow either:

- The default Seldon protocol. 
- The [KFServing V2 protocol](https://docs.seldon.io/projects/seldon-core/en/latest/servers/xgboost.html##v2-kfserving-protocol-incubating).

For details on each of these protocols, you can check the [documentation section on API protocols](https://docs.seldon.io/projects/seldon-core/en/latest/graph/protocols.html#v2-kfserving-protocol).

### Default Seldon protocol

We can deploy a XGBoost model uploaded to an object store by using the XGBoost model server implementation as shown in the config below:

In [7]:
%%writefile resources/iris.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: xgboost
spec:
  name: iris
  predictors:
  - graph:
      children: []
      implementation: XGBOOST_SERVER
      modelUri: gs://seldon-models/xgboost/iris
      name: classifier
    name: default
    replicas: 1

Writing resources/iris.yaml


And then we apply it to deploy it to our kubernetes cluster

In [8]:
!kubectl apply -f resources/iris.yaml

seldondeployment.machinelearning.seldon.io/xgboost created


In [9]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=xgboost -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "xgboost-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "xgboost-default-0-classifier" successfully rolled out


#### Rest Requests

In [10]:
X=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0, 6.0]]}}' \
   -X POST http://localhost:8003/seldon/seldon/xgboost/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

{'data': {'names': [], 'ndarray': [2.0]}, 'meta': {}}


In [11]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="xgboost",namespace="seldon")

In [12]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 4
    values: 0.617330874549778
    values: 0.4955768380888388
    values: 0.8325074250157055
    values: 0.29623988728724804
  }
}

Response:
{'data': {'names': [], 'tensor': {'shape': [1], 'values': [0.0]}}, 'meta': {}}


#### gRPC Requests

In [13]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1,4))
print(r)
assert(r.success==True)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 4], 'values': [0.7029137333313852, 0.9422923262038959, 0.7271094013077166, 0.18443703566724434]}}}
Response:
{'meta': {}, 'data': {'tensor': {'shape': [1], 'values': [0.0]}}}


In [14]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0,6.0]]}}' \
         -rpc-header seldon:xgboost -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)

{'meta': {}, 'data': {'ndarray': [2]}}


And delete the model we deployed

In [15]:
!kubectl delete -f resources/iris.yaml

seldondeployment.machinelearning.seldon.io "xgboost" deleted


### KFServing V2 protocol

We can deploy a XGBoost model, exposing an API compatible with [KFServing's V2 Protocol](https://docs.seldon.io/projects/seldon-core/en/latest/servers/xgboost.html##v2-kfserving-protocol-incubating) by specifying the `protocol` of our `SeldonDeployment` as `kfserving`.
For example, we can consider the config below:

In [1]:
%%writefile ./resources/iris-xgboost-v2.yaml
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: xgboost
spec:
  name: iris
  protocol: kfserving
  predictors:
  - graph:
      children: []
      implementation: XGBOOST_SERVER
      modelUri: gs://seldon-models/xgboost/iris
      name: classifier
    name: default
    replicas: 1

Overwriting ./resources/iris-xgboost-v2.yaml


We can then apply it to deploy our model to our Kubernetes cluster.

In [3]:
!kubectl apply -f ./resources/iris-xgboost-v2.yaml

seldondeployment.machinelearning.seldon.io/xgboost created


In [4]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=xgboost -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "xgboost-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "xgboost-default-0-classifier" successfully rolled out


Once it's deployed, we can send inference requests to our model.
Note that, since it's using the KFServing's V2 Protocol, these requests will be different to the ones using the default Seldon Protocol.

In [5]:
import requests
import json

inference_request = {
    "inputs": [
        {
          "name": "predict",
          "shape": [1, 4],
          "datatype": "FP32",
          "data": [[1, 2, 3, 4]]
        }
    ]
}

endpoint = "http://localhost:8003/seldon/seldon/xgboost/v2/models/infer"
response = requests.post(endpoint, json=inference_request)

print(json.dumps(response.json(), indent=2))
assert response.ok

{
  "model_name": "classifier",
  "model_version": "v1",
  "id": "d1887e45-e8c3-4644-b72b-5a3148463917",
  "parameters": null,
  "outputs": [
    {
      "name": "predict",
      "shape": [
        1
      ],
      "datatype": "FP32",
      "parameters": null,
      "data": [
        2.0
      ]
    }
  ]
}


Finally, we can delete the model we deployed.

In [6]:
!kubectl delete -f ./resources/iris-xgboost-v2.yaml

seldondeployment.machinelearning.seldon.io "xgboost" deleted


## Serve Tensorflow MNIST Model
We can deploy a tensorflow model uploaded to an object store by using the
tensorflow model server implementation as the config below.

This notebook contains two examples, one which shows how you can use the
TFServing prepackaged serve with the Seldon Protocol, and a second one which
shows how you can deploy it using the tensorlfow protocol (so you can send
requests of the exact format as you would to a tfserving server).

### Serve Tensorflow MNIST Model with Seldon Protocol

The config file below shows how you can deploy your Tensorflow model which
exposes the Seldon protocol.

In [7]:
%%writefile ./resources/mnist_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: tfserving
spec:
  name: mnist
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/mnist-model
      name: mnist-model
      parameters:
        - name: signature_name
          type: STRING
          value: predict_images
        - name: model_name
          type: STRING
          value: mnist-model
        - name: model_input
          type: STRING
          value: images
        - name: model_output
          type: STRING
          value: scores     
    name: default
    replicas: 1

Overwriting ./resources/mnist_rest.yaml


In [8]:
!kubectl apply -f ./resources/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io/tfserving created


In [9]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=tfserving -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "tfserving-default-0-mnist-model" rollout to finish: 0 of 1 updated replicas are available...
deployment "tfserving-default-0-mnist-model" successfully rolled out


In [10]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="tfserving",namespace="seldon")

#### REST Request

In [11]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,784))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 784
    values: 0.5360208678294703
    values: 0.5063574371425368
    values: 0.9443243610713166
    values: 0.0011084051211286416
    values: 0.20099891756759192
    values: 0.30071217406011774
    values: 0.15622672261211623
    values: 0.032474907686516064
    values: 0.7440398045429419
    values: 0.7108256047143693
    values: 0.7645229494651484
    values: 0.457444409411902
    values: 0.316314894432161
    values: 0.7350234986133874
    values: 0.6847025510931343
    values: 0.3124896535821692
    values: 0.5023732148295806
    values: 0.7766729114301516
    values: 0.31117856330829097
    values: 0.145499075089073
    values: 0.9955898498586168
    values: 0.04865057935951689
    values: 0.2557675256095512
    values: 0.3312323613365653
    values: 0.5179019306893689
    values: 0.2909692479700965
    values: 0.6482350619102406
    values: 0.23776475362315974
    values: 0.9684044714121269
    val

#### gRPC Request

In [12]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1,784))
print(r)
assert(r.success==True)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 784], 'values': [0.47209083184647593, 0.8838373150382969, 0.9181145850457837, 0.24859620806264326, 0.23293720880908908, 0.4941481171884624, 0.10725857979635278, 0.5287047511828012, 0.19695290280618072, 0.10247878859133952, 0.9738834180569024, 0.21298312177597456, 0.7460646984755017, 0.7849562914857755, 0.5704135117601201, 0.02792477113467806, 0.7793940787278147, 0.606568611346314, 0.29803598789024754, 0.1326626326173992, 0.32915350266835375, 0.0883570944110672, 0.7607531788790823, 0.1623233904716811, 0.5821722444997973, 0.6970934469577975, 0.35932488456489753, 0.06432456300653788, 0.9265409211003779, 0.38557298397196615, 0.9963670945752263, 0.37694931788140484, 0.16640281034611637, 0.9045008098130327, 0.305694315056507, 0.7073745635364587, 0.23511637628623283, 0.4853187768831849, 0.8487579535826453, 0.9742161341598627, 0.2840885206281021, 0.9946856297748927, 0.20052979354792833, 0.7052308229990519, 0.781644578

And delete the model we deployed

In [13]:
!kubectl delete -f ./resources/mnist_rest.yaml

seldondeployment.machinelearning.seldon.io "tfserving" deleted


### Serve Tensorflow Model with Tensorflow protocol

The config file below shows how you can deploy your Tensorflow model which
exposes the Tensorflow protocol.

In [14]:
%%writefile ./resources/halfplustwo_rest.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: hpt
spec:
  name: hpt
  protocol: tensorflow
  transport: rest
  predictors:
  - graph:
      children: []
      implementation: TENSORFLOW_SERVER
      modelUri: gs://seldon-models/tfserving/half_plus_two
      name:  halfplustwo
      parameters:
        - name: model_name
          type: STRING
          value: halfplustwo
    name: default
    replicas: 1

Overwriting ./resources/halfplustwo_rest.yaml


In [15]:
!kubectl apply -f ./resources/halfplustwo_rest.yaml

seldondeployment.machinelearning.seldon.io/hpt created


In [16]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=hpt -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "hpt-default-0-halfplustwo" rollout to finish: 0 of 1 updated replicas are available...
deployment "hpt-default-0-halfplustwo" successfully rolled out


In [17]:
import json
X=!curl -s -d '{"instances": [1.0, 2.0, 5.0]}' \
   -X POST http://localhost:8003/seldon/seldon/hpt/v1/models/halfplustwo/:predict \
   -H "Content-Type: application/json"
d=json.loads("".join(X))
print(d)
assert(d["predictions"][0] == 2.5)

{'predictions': [2.5, 3.0, 4.5]}


In [18]:
X=!cd ../executor/proto && grpcurl \
   -d '{"model_spec":{"name":"halfplustwo"},"inputs":{"x":{"dtype": 1, "tensor_shape": {"dim":[{"size": 3}]}, "floatVal" : [1.0, 2.0, 3.0]}}}' \
   -rpc-header seldon:hpt -rpc-header namespace:seldon \
   -plaintext -proto ./prediction_service.proto \
   0.0.0.0:8003 tensorflow.serving.PredictionService/Predict
d=json.loads("".join(X))
print(d)
assert(d["outputs"]["x"]["floatVal"][0] == 2.5)

{'outputs': {'x': {'dtype': 'DT_FLOAT', 'tensorShape': {'dim': [{'size': '3'}]}, 'floatVal': [2.5, 3, 3.5]}}, 'modelSpec': {'name': 'halfplustwo', 'version': '123', 'signatureName': 'serving_default'}}


In [19]:
!kubectl delete -f ./resources/halfplustwo_rest.yaml

seldondeployment.machinelearning.seldon.io "hpt" deleted


## Serve MLFlow Elasticnet Wines Model
We can deploy an MLFlow model uploaded to an object store by using the MLFlow
model server implementation as the config below:

In [20]:
%%writefile ./resources/elasticnet_wine.yaml
apiVersion: machinelearning.seldon.io/v1alpha2
kind: SeldonDeployment
metadata:
  name: mlflow
spec:
  name: wines
  predictors:
  - componentSpecs:
    - spec:
        # We are setting high failureThreshold as installing conda dependencies
        # can take long time and we want to avoid k8s killing the container prematurely
        containers:
        - name: classifier
          livenessProbe:
            initialDelaySeconds: 80
            failureThreshold: 200
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
          readinessProbe:
            initialDelaySeconds: 80
            failureThreshold: 200
            periodSeconds: 5
            successThreshold: 1
            httpGet:
              path: /health/ping
              port: http
              scheme: HTTP
    graph:
      children: []
      implementation: MLFLOW_SERVER
      modelUri: gs://seldon-models/mlflow/elasticnet_wine
      name: classifier
    name: default
    replicas: 1

Overwriting ./resources/elasticnet_wine.yaml


In [21]:
!kubectl apply -f ./resources/elasticnet_wine.yaml

seldondeployment.machinelearning.seldon.io/mlflow created


In [22]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=mlflow -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "mlflow-default-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "mlflow-default-0-classifier" successfully rolled out


### REST requests

In [24]:
X=!curl -s -d '{"data": {"ndarray":[[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1]]}}' \
   -X POST http://localhost:8003/seldon/seldon/mlflow/api/v1.0/predictions \
   -H "Content-Type: application/json"
d=json.loads(X[0])
print(d)

{'data': {'names': [], 'ndarray': [5.275558760255382]}, 'meta': {}}


In [25]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="mlflow",namespace="seldon")

In [26]:
r = sc.predict(gateway="ambassador",transport="rest",shape=(1,11))
print(r)
assert(r.success==True)

Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 11
    values: 0.358817295772466
    values: 0.42774982123748906
    values: 0.7525229974243451
    values: 0.4837952054509831
    values: 0.03971051054852981
    values: 0.7977411786519535
    values: 0.843901872160243
    values: 0.5876114963145197
    values: 0.6716037188345559
    values: 0.8544908465852457
    values: 0.3759460042213313
  }
}

Response:
{'data': {'names': [], 'tensor': {'shape': [1], 'values': [5.2069715754680725]}}, 'meta': {}}


### gRPC Requests

In [27]:
X=!cd ../executor/proto && grpcurl -d '{"data":{"ndarray":[[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1]]}}' \
         -rpc-header seldon:mlflow -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict
d=json.loads("".join(X))
print(d)

{'meta': {}, 'data': {'ndarray': [5.275558760255382]}}


In [28]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1,11))
print(r)
assert(r.success==True)

Success:True message:
Request:
{'meta': {}, 'data': {'tensor': {'shape': [1, 11], 'values': [0.35040541408334647, 0.9104661327491711, 0.4362897953086141, 0.07252654389819702, 0.5452850097930405, 0.8161583853649494, 0.9842675249693481, 0.6805591646597872, 0.6430042482083499, 0.7862682374046126, 0.5803183012135262]}}}
Response:
{'meta': {}, 'data': {'tensor': {'shape': [1], 'values': [5.226165754389033]}}}


In [29]:
!kubectl delete -f ./resources/elasticnet_wine.yaml

seldondeployment.machinelearning.seldon.io "mlflow" deleted
